# Dynamic Research Tutorial

This notebook demonstrates an end-to-end research workflow using `ResearchManager`.

In [ ]:
from omicverse.llm.dr import ResearchManager

class DummyStore:
    def search(self, query):
        class Doc:
            def __init__(self, id, text):
                self.id=id
                self.text=text
        return [Doc(1, f"background on {query}")]

rm = ResearchManager(vector_store=DummyStore())
brief = rm.scope("Example project")
findings = rm.research(brief)
report = rm.write(brief, findings)
print(report)
